# Basic Geopandas Examples
https://geopandas.org/en/stable/index.html
https://geopandas.org/en/stable/docs/user_guide/mapping.html

Frank Donnelly
Head of GIS & Data Services, Brown University Library

Dec 11, 2024 / revised Mar 2, 2025

## Brown Univ CoLab Users
1. Open this notebook with this URL:
https://colab.research.google.com/github/Brown-University-Library/geodata_geopandas_basic/blob/main/geopandas_example.ipynb

2. Then run the following box to import this repo into a temporary folder:

In [ ]:
# GOOGLE COLAB USERS - RUN THIS
!git clone https://github.com/Brown-University-Library/geodata_geopandas_basic temp_repo && mv temp_repo/* temp_repo/.[!.]* . && rm -rf temp_repo

## Import Modules

In [ ]:
import os, pandas as pd
import geopandas as gpd
from shapely.geometry import LineString
import matplotlib.pyplot as plt
%matplotlib inline

## Read Input and Create Geometry From Points
Must know and plot the CRS of points as is; CRS is referenced using EPSG codes: https://epsg.io/. 4269 is NAD 83, often used by US federal govt. The most common long / lat CRS is 4326, WGS 84.

In [ ]:
# Read shapefile
county_file=os.path.join('input','ri_county_bndy.shp')
gdf_cnty=gpd.read_file(county_file)
gdf_cnty.head()

In [ ]:
# Basic plot with GeoPandas
gdf_cnty.plot()

In [ ]:
# Read CSV
point_file=os.path.join('input','test_points.csv')
df_pnts=pd.read_csv(point_file, index_col='OBS_NUM', delimiter=',',dtype={'GEOID':str})

gdf_pnts = gpd.GeoDataFrame(df_pnts,geometry=gpd.points_from_xy(
    df_pnts['INTPTLONG'],df_pnts['INTPTLAT']),crs = 'EPSG:4269')
gdf_pnts

In [ ]:
gdf_pnts.plot()

## Get CRS Info and Transform
CRS Library: https://epsg.io/. 4269 is NAD 83, often used by US federal govt. The most common long / lat CRS is 4326, WGS 84.

In [ ]:
# CRS Counties
gdf_cnty.crs

In [ ]:
# Bounds Counties
gdf_cnty.total_bounds

In [ ]:
# CRS Points
gdf_pnts.crs

In [ ]:
# Bounds Points
gdf_pnts.total_bounds

In [ ]:
# Transform Point Geometry to Match County CRS
# 3438 is the EPSG for for the RI State Plane Zone
gdf_pnts.to_crs(3438,inplace=True)
gdf_pnts.crs

In [ ]:
gdf_pnts.total_bounds

In [ ]:
basemap = gdf_cnty.plot(color='white', edgecolor='black')

gdf_pnts.plot(ax=basemap, color='red');

## Spatial Join Points and Polygons
Keep all points on left, null values for non-matching counties on right. Take subset of columns from right, must always include geom for spatial joins.

In [ ]:
gdf_pnts_wcnty=gpd.sjoin(gdf_pnts, gdf_cnty[['geoid','namelsad','geometry']],
                         how='left', predicate='intersects')
gdf_pnts_wcnty.rename(columns={'geoid': 'COUNTY_ID', 'namelsad': 'COUNTY'}, inplace=True)
gdf_pnts_wcnty.loc[:,['OBS_NAME','OBS_DATE','COUNTY']]

Count the number of points in each polygon.

In [ ]:
cnty_count=gdf_pnts_wcnty.groupby(['COUNTY_ID','COUNTY'],dropna=False).agg(numpoints=('COUNTY_ID', 'count'))
cnty_count

## Generate Lines from Points Grouped by Categories and Calculate Length
Assumes points are in proper sequence; if not, sort by sequence column first. Output measurement units are based on the CRS.

In [ ]:
lines = gdf_pnts.groupby('GROUP')['geometry'].apply(lambda x: LineString(x.tolist()))
gdf_lines = gpd.GeoDataFrame(lines, geometry='geometry',crs = 'EPSG:3438').reset_index()
gdf_lines['length_mi']=(gdf_lines.length)/5280
gdf_lines.loc[:,['GROUP','length_mi']]

## Plot

In [ ]:
# Use matplotlib for more detailed plot configuration
fig, ax = plt.subplots()
plt.xticks(rotation=315)
gdf_cnty.plot(ax=ax, color='white', edgecolor='grey')
gdf_pnts.plot(ax=ax,color='black', markersize=5)
gdf_lines.plot(ax=ax, column="GROUP", legend=True)

## WRITE OUTPUT
Shapefile format cannot handle column names > 10 characters

In [ ]:
out_points=os.path.join('output','test_points_counties.shp')
out_lines=os.path.join('output','test_lines.shp')

gdf_pnts_wcnty.to_file(out_points)
gdf_lines.to_file(out_lines)